# Langchain Agent

In [1]:
from langchain.prompts.prompt import PromptTemplate
from langchain.agents import initialize_agent, Tool, create_react_agent, AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.utilities import GoogleSerperAPIWrapper
from langchain_openai import AzureChatOpenAI

deployment = "gpt-4o"

llm = AzureChatOpenAI(deployment_name=deployment, model_name=deployment, temperature=0.8, max_tokens=256)


def query_web(question):
    search = GoogleSerperAPIWrapper()
    return search.run(question)


def get_current_cluster_state(cluster_name):
    print(f"cluster:{cluster_name}")
    return """ERROR: Failed to pull image "chaocai/docker/dsp:latest"""


tools = [
    Tool(
        name="Get current info",
        func=query_web,
        description="""invoke it for when you need to answer questions about current info.
             And the input should be a search query."""),
    Tool(
        name="Get current cluster state",
        func=get_current_cluster_state,
        description="""invoke it for when you need to get the current state in a given cluster, 
        And the input should be service name.""")
]

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, max_iterations=3, verbose=True)


template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

agent2 = create_react_agent(llm=llm, tools=tools, prompt=prompt)
def _handle_error(error) -> str:
    return str(error)[:50]


agent_executor = AgentExecutor(
    agent=agent2,
    tools=tools,
    verbose=True,
    handle_parsing_errors=_handle_error,
    # handle_parsing_errors=True
    
)




/Users/gevin/projects/pycharm/ai/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [2]:
# agent.invoke("杭州今天穿什么合适？")
agent_executor.invoke({"input": "杭州今天穿什么合适？"})
# agent2.invoke("杭州今天穿什么合适？")
# agent2. invoke(input="杭州今天穿什么合适？")



> Entering new AgentExecutor chain...
To determine what is suitable to wear in Hangzhou today, I need to check the current weather conditions in Hangzhou.

Action: Get current info
Action Input: "Hangzhou weather today"71°FObservation alone is not enough; I need to understand the weather conditions such as temperature, precipitation, and wind to give a comprehensive recommendation on what to wear.

Action: Get current info
Action Input: "current weather conditions in Hangzhou"71°FI need more specific details about the weather conditions like precipitation, wind speed, and whether it is sunny or cloudy to provide a complete recommendation.

Action: Get current info
Action Input: "current weather forecast for Hangzhou"71°FIt seems that I am repeatedly getting the same temperature information without the necessary additional details. Instead of continuing with the same approach, I should provide some general advice based on the given temperature of 71°F (around 22°C), which is moderate.

{'input': '杭州今天穿什么合适？',
 'output': 'With the current temperature in Hangzhou being around 71°F (22°C), it is comfortable to wear light to medium-weight clothing. You might consider wearing a t-shirt or blouse with jeans or pants. Carrying a light jacket or sweater is also advisable, especially for the evening when temperatures may drop.'}

In [16]:
agent_executor.invoke({"input": "如何修复的ADX集群？"})
# agent_executor.invoke({"input": "What's wrong with the cluster 'ADX'? And if there's an error, give me some suggestion. "})



> Entering new AgentExecutor chain...
To answer the question "如何修复的ADX集群？" (How to repair an ADX cluster?), I need to gather specific information about the current state of the ADX cluster. 

Action: Get current cluster state
Action Input: ADX cluster
cluster:ADX cluster

ERROR: Failed to pull image "chaocai/docker/dsp:latestIt seems that there was an error in retrieving the current state of the ADX cluster. This error might indicate issues with the infrastructure or the service itself. Let's break down the approach to repair an ADX (Azure Data Explorer) cluster:

1. **Check Cluster State**: Start by examining the current state of the ADX cluster. This includes looking for any status messages or alerts.

2. **Review Logs**: Check the logs for any anomalies or specific error messages that can give clues on what might be wrong.

3. **Restart Services**: Sometimes, simply restarting the services can resolve transient issues.

4. **Check Resource Utilization**: Ensure that the cluster ha

{'input': '如何修复的ADX集群？',
 'output': 'To repair an ADX (Azure Data Explorer) cluster, follow these steps:\n1. Check the current'}

In [20]:
!pip install wolframalpha

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.5 kB/s eta 0:00:00a 0:00:02


In [22]:
from langchain.utilities.wolfram_alpha import WolframAlphaAPIWrapper
def wolfram_api(question):
    wolfram = WolframAlphaAPIWrapper()
    return wolfram.run(question)

In [23]:
wolfram_api("What is 2x+5 = -3x + 7?")

ValidationError: 1 validation error for WolframAlphaAPIWrapper
__root__
  Did not find wolfram_alpha_appid, please add an environment variable `WOLFRAM_ALPHA_APPID` which contains it, or pass `wolfram_alpha_appid` as a named parameter. (type=value_error)

In [24]:
tools = [
    Tool(
        name = "Get current info",func=query_web,
        description="""invoke it for when you need to answer questions about current info.
             And the input should be a search query."""),
    Tool(
        name="Get current cluster state", func=get_current_cluster_state,
        description="""invoke it for when you need to get the current state in a given cluster, 
        And the input should be service name."""),
    Tool(
        name="Invoke WolframAlpha API", func=wolfram_api,
        description="""invoke it for when you need to anwser or solve a mathematical question, 
        And the input is the mathematical equations""")
]

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, max_iterations=10, verbose=True)

In [25]:
agent.invoke("在一个外星球上，这里的鸡都长着3只脚1个头，免子有4只脚1个头，一个装着鸡和免子的笼子里一共有20个头，71只脚，请问笼子里各有多少只鸡和免子？")



> Entering new AgentExecutor chain...
这个问题可以看作是一个包含两个变量的线性方程组问题。我们可以设鸡的数量为\( x \)，兔子的数量为\( y \)。根据题意，可以列出以下两个方程：

1. \( x + y = 20 \)（头的总数）
2. \( 3x + 4y = 71 \)（脚的总数）

接下来，我们将使用WolframAlpha API来解决这个方程组。

Action: Invoke WolframAlpha API
Action Input: solve x + y = 20, 3x + 4y = 71

ValidationError: 1 validation error for WolframAlphaAPIWrapper
__root__
  Did not find wolfram_alpha_appid, please add an environment variable `WOLFRAM_ALPHA_APPID` which contains it, or pass `wolfram_alpha_appid` as a named parameter. (type=value_error)